In [1]:
from codebase.classes import Particles
from codebase.classes_data import Data
import pandas as pd
import numpy as np
from codebase.file_utils import (
    save_obj,
    load_obj,
    make_folder,
    path_backslash
)
from codebase.ibis import essl, exp_and_normalise, model_phonebook, run_mcmc
from codebase.plot import get_post_df, plot_density
from codebase.run_tlk import model_phonebook
from tqdm import tqdm
from scipy.special import logsumexp
import altair as alt
alt.data_transformers.disable_max_rows()


import pystan

## Load Model Evidence

In [2]:
log_dirs = dict()
log_dirs['m6'] = 'log/fabian_real_muthen/20210829_151703_minibig5_m6/'
log_dirs['m11'] = 'log/fabian_real_muthen/20210829_151657_minibig5_m11/'
log_dirs['m10'] = 'log/fabian_real_muthen/20210829_151659_minibig5_m10/'
log_dirs['m9'] = 'log/fabian_real_muthen/20210829_151702_minibig5_m9/'

# load existing results
bf = dict()
for model in [6,9,10,11]:
    bf['m'+str(model)] = load_obj('log_lklhds', log_dirs['m'+str(model)])


In [3]:
bfdf = pd.DataFrame.from_dict(bf)

bfdf['t']  = np.arange(len(bfdf))
for i in [9,10,11]:
    bfdf['m6/m'+str(i)] = bfdf['m6']/bfdf['m'+str(i)]
bfdf.drop(['m6', 'm9', 'm10', 'm11'], axis=1, inplace=True)
bfdf = bfdf.melt(id_vars='t', var_name='model')
bfdf = bfdf[bfdf.t>=30]

INFO:numexpr.utils:NumExpr defaulting to 8 threads.


In [4]:
width = 600
height = 400
c = alt.Chart(bfdf).mark_line(
    strokeWidth = 1,
    ).encode(
    alt.X('t:Q', title=None),
    alt.Y('value:Q'),
    alt.Color('model')
    ).properties(width=width, height=height).interactive()
c

alt.Chart(...)

## Scoring Rules


In [5]:
log_dirs = dict()
log_dirs['m6'] = 'log/fabian_real_muthen/20210829_151703_minibig5_m6/'
log_dirs['m11'] = 'log/fabian_real_muthen/20210829_151657_minibig5_m11/'
log_dirs['m10'] = 'log/fabian_real_muthen/20210829_151659_minibig5_m10/'
log_dirs['m9'] = 'log/fabian_real_muthen/20210829_151702_minibig5_m9/'

# load existing results
scrl = dict()
for model in [6,9,10,11]:
    scrl['m'+str(model)] = load_obj('scoring_rule', log_dirs['m'+str(model)])


In [6]:
scrldf = pd.DataFrame.from_dict(bf)

scrldf['t']  = np.arange(len(scrldf))
# for i in [9,10,11]:
#     scrldf['m6/m'+str(i)] = scrldf['m6']/scrldf['m'+str(i)]
# scrldf.drop(['m6', 'm9', 'm10', 'm11'], axis=1, inplace=True)
scrldf = scrldf.melt(id_vars='t', var_name='model')
scrldf = scrldf[scrldf.t>=30]
scrldf['cval'] = scrldf.value.cumsum()

In [7]:
width = 600
height = 400
c = alt.Chart(scrldf).mark_line(
    strokeWidth = 1,
    ).encode(
    alt.X('t:Q', title=None),
    alt.Y('cval:Q'),
    alt.Color('model')
    ).properties(width=width, height=height).interactive()
c

alt.Chart(...)